In [1]:
import os, sys
import numpy as np
import pandas as pd
import pickle
import random
import matplotlib.pyplot as plt
import pylab

import itertools
from itertools import product
import json

from shutil import copyfile


In [2]:
def make_units_with_lyr_for_cstr(expt_id, units_fname, cstr):
    
    cstr_lyr = pd.DataFrame()
    #units_fname = os.path.join(units_df_basedir,expt_id + '_units_df.pkl')
    units_df = pd.read_pickle(units_fname)
    visp_units = units_df[units_df.ecephys_structure_acronym == cstr].index.values#[:5]
    
    lyrs_dir = '/Users/Ram/Dropbox/VC_NP_sklearn_copy/layer_est_from_Jung/'
    lyr_fname = os.path.join(lyrs_dir,expt_id+'_layer.json')
    try:
        with open(lyr_fname) as datafile:
            data = json.load(datafile)
        lyr_df = pd.DataFrame(data.items(), index = data.keys())
        cstr_lyr = lyr_df.loc[list(units_df[units_df.ecephys_structure_acronym ==cstr].peak_channel_id.values.astype(str))]
        cstr_lyr.columns = ['peak_channel_id','layer']
        cstr_lyr['unit_id'] = visp_units
    except Exception as e:
        print(e)
        
    return units_df, cstr_lyr

def get_scale_fnames_with_max_test_corr_wvis(prs_df_dir, expt_id, cstr, unit_id):
    
    separator = '_'
    
    cstr_scale = 1
    stim_scale_list = [0.0, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0]
    tmp = pd.DataFrame()
    for stim_scale in stim_scale_list:
        
        fname = separator.join([str(unit_id), 'fr_scale',str(cstr_scale), \
                                                  'stim_scale', str(stim_scale)]) + '.pkl'
        
        if os.path.isfile(os.path.join(prs_df_dir,fname)):
            tmp_pd = pd.read_pickle(os.path.join(prs_df_dir,fname))
            #tmp_pd.insert(loc=4,column='scale',value = scale)
            tmp = pd.concat((tmp,tmp_pd),sort=False)
    
    max_ind = np.argmax(tmp.test_corr.values)
    max_scale = tmp.iloc[max_ind]['stim_scale']
    max_fname = separator.join([str(tmp.iloc[max_ind]['unit_id']), 'fr_scale',str(cstr_scale), \
                                                  'stim_scale', str(max_scale)]) + '.pkl' 
    #print(unit_id, max_scale, tmp.test_corr.values)
    
    return max_fname

#    return tmp

In [3]:
units_df_basedir = '/Users/Ram/Dropbox/VC_NP_sklearn_copy/units_dfs'
basedir = '/Users/Ram/Dropbox/VC_NP_sklearn_mod/sandbox_save_prs'

expt_id_list = ['715093703']#['719161530']#['715093703'] #,'719161530'
split_frac = 0.5

stim_name = 'static_gratings'
bin_dt = 0.005
bin_start = 0.05#0.05
bin_end = 0.15#0.15

vis_cols = ['VISp']#['LGd','LP','VISam','VISpm','VISp','VISl','VISrl']
for cstr in vis_cols:
    for expt_id in expt_id_list:

        units_fname = os.path.join(units_df_basedir,expt_id + '_units_df.pkl')
        units_df, cstr_lyr_df = make_units_with_lyr_for_cstr(expt_id, units_fname, cstr)
        
        print(ram)
        
        visp_units = units_df[units_df.ecephys_structure_acronym == cstr].index.values

        prs_df_dir =  os.path.join(basedir, expt_id, cstr, \
           stim_name, 't1_'+str(bin_start) + '_t2_'+str(bin_end) + '_dt_'+str(bin_dt))
        
        max_fname_list = []
        for unit_id in visp_units:
            max_fname = get_scale_fnames_with_max_test_corr_wvis(prs_df_dir, expt_id, cstr, unit_id)
            max_fname_list.append(max_fname)
        
        src_dir = prs_df_dir
        dst_dir = os.path.join(prs_df_dir,'max_test_corr')
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        for max_fname in max_fname_list:
            src_fname = os.path.join(src_dir,max_fname)
            dst_fname = os.path.join(dst_dir, max_fname)
            copyfile(src_fname, dst_fname)

NameError: name 'ram' is not defined

In [ ]:
#max_fname_list

In [5]:
cstr_lyr_df.sort_values('layer')

,peak_channel_id,layer,unit_id
850261846,850261846,2,950933960
850261814,850261814,2,950932445
850261838,850261838,2,950932696
850261822,850261822,2,950932563
850261822,850261822,2,950932578
...,...,...,...
850261718,850261718,6,950930375
850261724,850261724,6,950930522
850261726,850261726,6,950933732
850261720,850261720,6,950930437


In [14]:
[(round(ii,2)) for ii in 0.05 + np.arange(0.000,0.251,0.050)]

[0.05, 0.1, 0.15, 0.2, 0.25, 0.3]